## Part 1: Data loading

In [2]:
import pandas as pd

In [3]:
# Load the dataset from CSV
graph_details = pd.read_csv(r'data/books_graph_facts.csv')

In [4]:
# Find the shape
graph_details.shape

(600000, 3)

## Part 2:  Handling Missing Data:

In [5]:
# Check for missing values
print(graph_details.isnull().sum())

head        0
relation    0
tail        0
dtype: int64


In [6]:
# Option 1: Remove rows with missing values
df_cleaned = graph_details.dropna()

In [7]:
# Option 2: Impute missing values (e.g., with the most frequent value or a placeholder)
df_cleaned = graph_details.fillna('Unknown')

In [8]:
df_cleaned.shape

(600000, 3)

## Part 3:  Removing Duplicates
Duplicate rows can introduce bias in the analysis. Here, we identify and remove all duplicate rows from the dataset.

In [17]:
# Find and display all duplicate rows in the DataFrame
duplicates = graph_details[graph_details.duplicated()]

# Check if there are any duplicates, and then display them
if not duplicates.empty:
    print("Duplicates found:")
    print(duplicates)
else:
    print("No duplicates found.")

Duplicates found:
              head          relation           tail
2951     book_4703  belongs_to_genre        genre_4
4655     book_1791  belongs_to_genre       genre_18
6048      book_346  belongs_to_genre       genre_12
6104     book_1904  belongs_to_genre        genre_1
6320     book_4202  belongs_to_genre        genre_1
...            ...               ...            ...
599950   book_3364  belongs_to_genre        genre_8
599960   book_4017  belongs_to_genre        genre_4
599961   book_3909  belongs_to_genre       genre_11
599967   book_3555      published_by  publisher_120
599983  reader_368              read      book_2938

[36705 rows x 3 columns]


In [8]:
# Remove duplicates
df_cleaned = graph_details.drop_duplicates()

In [16]:
df_cleaned[(df_cleaned['head'] == 'book_4703') & (df_cleaned['relation'] == 'belongs_to_genre') & (df_cleaned['tail'] == 'genre_4')]

,head,relation,tail
2803,book_4703,belongs_to_genre,genre_4


## Part 4:  To ensure data consistency

In [18]:
# Display the data types of all columns in the DataFrame
print(df_cleaned.dtypes)

head        object
relation    object
tail        object
dtype: object


### a. Data Consistency: Ensuring Consistent Naming
To avoid inconsistencies, this step standardizes all text data to lowercase.

In [21]:
# Standardizing text data to lowercase for consistency
df_cleaned = df_cleaned.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df_cleaned

,head,relation,tail
0,book_3725,won_award,award_85
1,book_1435,published_by,publisher_152
2,author_944,wrote,book_3099
3,reader_4577,read,book_2631
4,book_1633,belongs_to_genre,genre_11
...,...,...,...
599995,reader_3389,read,book_3783
599996,book_1968,belongs_to_genre,genre_10
599997,reader_1680,read,book_4606
599998,book_4405,won_award,award_35


### b. Outlier Detection: Detecting and Handling Outliers - Identifying Entities with Unusual Relations
Using Z-scores, this section identifies entities with abnormally high or low relationships, which may indicate anomalies in the data.

In [23]:
import numpy as np

# Calculate the degree of each entity (how many relations it participates in)
entity_degrees = pd.concat([df_cleaned['head'], df_cleaned['tail']]).value_counts()

In [24]:
# Calculate Z-scores to detect outliers (outliers in entity connectivity)
mean_degree = np.mean(entity_degrees)
std_degree = np.std(entity_degrees)
z_scores = (entity_degrees - mean_degree) / std_degree

In [29]:
# Set threshold for outliers (e.g., Z-score > 3 or < -3)
outliers = entity_degrees[(z_scores.abs() > 3)]

In [32]:
# Display the outliers
print("Entities with unusually high or low number of relations (outliers):")
print(outliers)

# Option 1: Flag outliers by creating a separate column in the DataFrame
df_cleaned['is_outlier'] = df_cleaned['head'].isin(outliers.index) | df_cleaned['tail'].isin(outliers.index)

# Option 2: Remove the outliers directly
df_cleaned = df_cleaned[~df_cleaned['head'].isin(outliers.index) & ~df_cleaned['tail'].isin(outliers.index)]

# Verify the cleaned DataFrame
print(df_cleaned.head())

Entities with unusually high or low number of relations (outliers):
genre_3      2942
genre_19     2926
genre_12     2912
genre_15     2907
genre_13     2897
             ... 
award_116     508
award_42      507
award_118     501
award_87      501
award_134     498
Name: count, Length: 169, dtype: int64
          head      relation           tail  is_outlier
1    book_1435  published_by  publisher_152       False
2   author_944         wrote      book_3099       False
3  reader_4577          read      book_2631       False
5  reader_2664          read       book_252       False
6   reader_444          read       book_319       False


In [33]:
df_cleaned.shape

(424775, 4)

In [35]:
# Replace the original DataFrame with the cleaned one
df = df_cleaned

# Optionally, save the cleaned DataFrame to a new file
df.to_csv('cleaned_books_graph_facts.csv', index=False)